In [1]:
%matplotlib inline

import sys
import os
import inspect

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from osgeo import gdal
import osr

import h5py

sys.path.append('D:/')
from my_packages.My_Geoprocess import*


In [89]:
def getNcPath(NetCdf_data_path):
    return [f for f in os.listdir(NetCdf_data_path) if f.endswith('.nc')]

NetCdf_data_path = "D:/Image/Poe/Acolyte/S2B_MSIL1C_20180320T230859_N0206_R101_T58KEB_20180321T002930/"
raster_data_path = getNcPath(NetCdf_data_path)

raster_dataset = gdal.Open(NetCdf_data_path+raster_data_path[0])
SubDataSet_md = raster_dataset.GetMetadata('SUBDATASETS')
ds = []

# Try opening the 4 subdatasets
for i in range(1,int(len(SubDataSet_md)/2)+1):
    gdal.ErrorReset()
    ds.append(gdal.Open(SubDataSet_md['SUBDATASET_%d_NAME' % i]))
    if ds is None or gdal.GetLastErrorMsg() != '':
        print('subdatasets failed to load')
        print(SubDataSet_md['SUBDATASET_%d_NAME' % i])
ds[0].RasterCount

1

In [90]:
raster_data_path

['S2B_MSI_2018_03_20_23_08_59_T58KEB_L1R.nc',
 'S2B_MSI_2018_03_20_23_08_59_T58KEB_L2R.nc',
 'S2B_MSI_2018_03_20_23_08_59_T58KEB_L2W.nc']

In [29]:
raster_dataset = gdal.Open(NetCdf_data_path+raster_data_path[1], gdal.GA_ReadOnly)
dataset = raster_dataset.GetSubDatasets()

for df in dataset:
    buffer = gdal.Open(df[0],gdal.GA_ReadOnly)
    print(buffer.RasterXSize, 'x', buffer.RasterYSize)

8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346
8412 x 5346


In [88]:
dataset

[('HDF5:"D:/Image/Poe/Acolyte/S2B_MSIL1C_20180320T230859_N0206_R101_T58KEB_20180321T002930/S2B_MSI_2018_03_20_23_08_59_T58KEB_L2R.nc"://lat',
  '[5346x8412] //lat (64-bit floating-point)'),
 ('HDF5:"D:/Image/Poe/Acolyte/S2B_MSIL1C_20180320T230859_N0206_R101_T58KEB_20180321T002930/S2B_MSI_2018_03_20_23_08_59_T58KEB_L2R.nc"://lon',
  '[5346x8412] //lon (64-bit floating-point)'),
 ('HDF5:"D:/Image/Poe/Acolyte/S2B_MSIL1C_20180320T230859_N0206_R101_T58KEB_20180321T002930/S2B_MSI_2018_03_20_23_08_59_T58KEB_L2R.nc"://rhos_1610',
  '[5346x8412] //rhos_1610 (64-bit floating-point)'),
 ('HDF5:"D:/Image/Poe/Acolyte/S2B_MSIL1C_20180320T230859_N0206_R101_T58KEB_20180321T002930/S2B_MSI_2018_03_20_23_08_59_T58KEB_L2R.nc"://rhos_2186',
  '[5346x8412] //rhos_2186 (64-bit floating-point)'),
 ('HDF5:"D:/Image/Poe/Acolyte/S2B_MSIL1C_20180320T230859_N0206_R101_T58KEB_20180321T002930/S2B_MSI_2018_03_20_23_08_59_T58KEB_L2R.nc"://rhos_442',
  '[5346x8412] //rhos_442 (64-bit floating-point)'),
 ('HDF5:"D:/Imag

In [51]:
'''(Uy, Ux) = np.mgrid[0:10,0:5]
(ny, nx) = Ux.shape
E0, Enx = Ux[0,0], Ux[0,nx-1]'''

In [84]:
ds[1].RasterXSize

8412

In [ ]:
write_raster2(FilePath+'/Mask2/'+'10m.tif', SoilIndex[:,:,0], ds[0].GetGeoTransform(), ds[0].GetProjection(), DriverName="GTiff", formatMem=gdal.GDT_Byte)

In [54]:
lines = inspect.getsource(write_raster2)
print(lines)

def write_raster2(fname, data, geo_transform, projection, DriverName="GTiff", formatMem=gdal.GDT_UInt16 ):
    """Create a GeoTIFF file with the given data."""
    
    driver = gdal.GetDriverByName(DriverName)
    if driver is None:
        print('DriverFailed')
    
    if len(data.shape)==3:
        rows, cols, NBands = data.shape
    else:
        rows, cols = data.shape
        NBands = 1

    dataset = driver.Create(fname, cols, rows,  NBands, formatMem )
    if driver is None:
        print('DatasetCreation Failed')

    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    if NBands > 1:
        for b in range(NBands): 
            band = dataset.GetRasterBand(b+1)
            band.WriteArray(data[:,:,b])
    else:
        band = dataset.GetRasterBand(1)
        band.WriteArray(data[:,:])
    dataset = None  # Close the file



In [55]:
def getNCGeoTrans2(file, LonS, LatS ):
    '''Extract Geotransform from Longitude and Latitude destination file'''
    
    f = h5py.File(file, 'r')
    long = f[LonS][:]
    lat = f[LatS][:]

    nx = len(long)
    ny = len(lat)
    xmin, ymin, xmax, ymax = [long.min(), lat.min(), long.max(), lat.max()]
    xres = (xmax - xmin) / float(nx)
    yres = (ymax - ymin) / float(ny)
    return (xmin, xres, 0, ymax, 0, -yres)



def getNCGeoTrans(ds_lon, ds_lat, epsg ):
    '''Extract Geotransform from Longitude and Latitude destination file'''
    
    longs = ds_lon.GetRasterBand(1).ReadAsArray()
    lats = ds_lat.GetRasterBand(1).ReadAsArray()
    proj_gcp = osr.SpatialReference()
    proj_gcp.ImportFromEPSG(epsg)
    transf = osr.CoordinateTransformation(proj_gcp, proj_out)
    ul = transf.TransformPoint(float(longs[0][0]), float(lats[0][0]))
    lr = transf.TransformPoint(float(longs[len(longs)-1][len(longs[0])-1]), float(lats[len(longs)-1][len(longs[0])-1]))
    ur = transf.TransformPoint(float(longs[0][len(longs[0])-1]), float(lats[0][len(longs[0])-1]))
    ll = transf.TransformPoint(float(longs[len(longs)-1][0]), float(lats[len(longs)-1][0]))
    gt0 = ul[0]
    gt1 = (ur[0] - gt0) / len(longs[0])
    gt2 = (lr[0] - gt0 - gt1*len(longs[0])) / len(longs)
    gt3 = ul[1]
    gt5 = (ll[1] - gt3) / len(longs)
    gt4 = (lr[1] - gt3 - gt5*len(longs) ) / len(longs[0])
    
    return (gt0,gt1,gt2,gt3,gt4,gt5)

datafile = NetCdf_data_path+raster_data_path[1]

LonS = 'lon'
LatS = 'lat'

datafile = NetCdf_data_path+raster_data_path[1]
BlueData = h5py.File(NetCdf_data_path+raster_data_path[1], 'r')['rhos_492']

projection = osr.SpatialReference()
projection.ImportFromEPSG(4326)

write_raster2('testscript.tif',
              BlueData, 
              getNCGeoTrans2(datafile, LonS, LatS), 
              projection.ExportToWkt(), DriverName="GTiff", 
              formatMem=gdal.GDT_CFloat64)

In [25]:
datafile = NetCdf_data_path+raster_data_path[1]
lon = h5py.File(NetCdf_data_path+raster_data_path[1], 'r')['lon'][:]

In [53]:
getNCGeoTrans2(datafile, LonS, LatS)

(165.00212122663831,
 0.00015221559775384353,
 0,
 -21.306798823565913,
 0,
 -9.0718149373187682e-05)

In [62]:
A = np.array([[1, nx, 0, 0, 0, 0],
             [1, 0, ny, 0, 0, 0],
             [0, 0, 0, 1, ny, 0],
             [0, 0, 0, 1, 0, nx],
             [1, 0, 0, 0, 0, 0],
             [0, 0, 0, 1, 0, 0]])
A.T

array([[ 1,  1,  0,  0,  1,  0],
       [ 5,  0,  0,  0,  0,  0],
       [ 0, 10,  0,  0,  0,  0],
       [ 0,  0,  1,  1,  0,  1],
       [ 0,  0, 10,  0,  0,  0],
       [ 0,  0,  0,  5,  0,  0]])

In [65]:
Enxny=7
Nnxny=8

A = np.array([[1, nx, 0, 0, 0, 0],[1, 0, ny, 0, 0, 0],[0, 0, 0, 1, 0, ny],[0, 0, 0, 1, nx, 0],[1, nx, ny, 0, 0, 0],[0, 0, 0, 1, nx, ny],[1, 0, 0, 0, 0, 0],[0, 0, 0, 1, 0, 0]])
C = np.array([Enx, Eny, Nny, Nnx, Enxny, Nnxny, E0, N0]).reshape(8,1)
A

array([[ 1,  5,  0,  0,  0,  0],
       [ 1,  0, 10,  0,  0,  0],
       [ 0,  0,  0,  1,  0, 10],
       [ 0,  0,  0,  1,  5,  0],
       [ 1,  5, 10,  0,  0,  0],
       [ 0,  0,  0,  1,  5, 10],
       [ 1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0]])

In [66]:
C

array([[1],
       [2],
       [3],
       [4],
       [7],
       [8],
       [5],
       [6]])

In [ ]:
bands_data = []
for f in range(len(raster_data_path)):
    raster_dataset = gdal.Open(rasters_data_path+raster_data_path[f], gdal.GA_ReadOnly)
    for b in range(1, raster_dataset.RasterCount+1):
        band = raster_dataset.GetRasterBand(b)
        bands_data.append(band.ReadAsArray())
        
for i in range(len(bands_data)):
    print(bands_data[i].shape)
bands_data = np.dstack(bands_data)
rows, cols, n_bands = bands_data.shape
print(bands_data.shape)

raster_dataset = gdal.Open(rasters_data_path+raster_data_path[0], gdal.GA_ReadOnly)
geo_transform = raster_dataset.GetGeoTransform()
proj = raster_dataset.GetProjectionRef()
del raster_dataset